# Deploy ML Model to SPCS Inference Service

This notebook deploys the TRANSFORMER_FAILURE_PREDICTOR model as a scalable SPCS inference service.

In [ ]:
import os
from snowflake.snowpark import Session
from snowflake.ml.registry import Registry

# Create session
connection_name = os.getenv("SNOWFLAKE_CONNECTION_NAME", "cpe_demo_CLI")
session = Session.builder.config("connection_name", connection_name).create()

print(f"Connected as: {session.get_current_user()}")
print(f"Account: {session.get_current_account()}")

In [ ]:
# Configuration
MODEL_DATABASE = "SI_DEMOS"
MODEL_SCHEMA = "ML_DEMO"
MODEL_NAME = "TRANSFORMER_FAILURE_PREDICTOR"
MODEL_VERSION = "V2_EXPLAINABLE"

SERVICE_NAME = "TRANSFORMER_ML_INFERENCE_SVC"
COMPUTE_POOL = "FLUX_INTERACTIVE_POOL"
MAX_INSTANCES = 2

# Set context
session.use_database(MODEL_DATABASE)
session.use_schema(MODEL_SCHEMA)

print(f"Database: {session.get_current_database()}")
print(f"Schema: {session.get_current_schema()}")

In [ ]:
# Initialize registry and get model
reg = Registry(session=session, database_name=MODEL_DATABASE, schema_name=MODEL_SCHEMA)

mv = reg.get_model(MODEL_NAME).version(MODEL_VERSION)
print(f"Model: {MODEL_NAME} version {MODEL_VERSION}")
print(f"Functions: {mv.show_functions()}")

In [ ]:
# Check existing services
existing = mv.list_services()
print(f"Existing services: {existing}")

if SERVICE_NAME in str(existing):
    print(f"\n⚠️ Service {SERVICE_NAME} already exists!")
else:
    print(f"\n✓ Ready to create service: {SERVICE_NAME}")

In [ ]:
# Deploy the inference service
print(f"Creating SPCS inference service...")
print(f"  Service: {SERVICE_NAME}")
print(f"  Compute Pool: {COMPUTE_POOL}")
print(f"  Max Instances: {MAX_INSTANCES}")

mv.create_service(
    service_name=SERVICE_NAME,
    service_compute_pool=COMPUTE_POOL,
    ingress_enabled=True,
    max_instances=MAX_INSTANCES
)

print(f"\n✓ Service creation initiated!")
print(f"  This typically takes 5-15 minutes.")

In [ ]:
# Check service status
services = session.sql(f"SHOW SERVICES LIKE '{SERVICE_NAME}' IN SCHEMA {MODEL_DATABASE}.{MODEL_SCHEMA}").collect()
for svc in services:
    print(f"Service: {svc['name']}")
    print(f"  Status: {svc['status']}")
    print(f"  Compute Pool: {svc['compute_pool']}")

In [ ]:
# Get endpoint URL when ready
try:
    endpoints = session.sql(f"SHOW ENDPOINTS IN SERVICE {MODEL_DATABASE}.{MODEL_SCHEMA}.{SERVICE_NAME}").collect()
    for ep in endpoints:
        print(f"Endpoint: {ep['name']}")
        print(f"  URL: {ep['ingress_url']}")
except Exception as e:
    print(f"Service not ready yet: {e}")